# PFG: Streaming API

Se realizan consultas en el flujo global de consultas de Twitter. No es histórico, sino en tiempo real.

## Carga inicial de datos

In [ ]:
import requests
from requests_oauthlib import OAuth1
url = 'https://stream.Twitter.com/1.1/statuses/filter.json'
auth = OAuth1(consumer_key, consumer_secret, access_token, access_token_secret)
pms = {'track' : 'premier league -filter:retweets AND -filter:replies', 'lang': 'en'}

Como con el REST_API se debe realizar la configuración de la información que se quiere rescatar en el Streaming, cargando la información necesaria del endpoint, las librerías, las claves y los tokens de acceso y los parámetros necesarios. 
La diferencia fundamental en la variable **pms** (la de los parámetros) es que, en este caso, al contrario que en REST_API, se indica como primer parámetro track y en REST_API se indicaba una consulta q.

In [ ]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client[database_name]
mongo = db[collection_name] #en REST_API el nombre de la variable era collection, pero aquí lo cambio por mongo para que coincida con la sección de código siguiente

<font color='red'>Para poder insertar la información en la base de datos se debe importar de nuevo la librería que permite utilizar Mongo. Seguramente no será igual que en el REST_API, pero lo pongo por ahora para después modificarlo.</font> Según la [documentación de mongo](https://docs.mongodb.com/manual/reference/method/db.collection.insert/) el método insert se utiliza en una colección de la base de datos, así que cambio el nombre de la variable collection por mongo.

In [ ]:
res = requests.post(url, auth=auth, params = pms, stream = True)
for line in res.iter_lines(): 
    if line:
        tweet = json.loads(line.decode('utf-8'))
        try:
            mongo.insert(tweet)
        except:
            pass

Esta sección de código se encarga de recibir la información.
Para ello en primer lugar se almacena en la variable **res** la información necesaria para la *consulta*: el endpoint, la información de autorización los parámetros e indicamos con una variable booleana que se trata de un Stream. 
Todo esto se hace mediante una petición post.
A continuación, mediante un bucle for se va iterando sobre cada línea que vaya devolviendo el post y por cada iteración se transforma la información en json (ya que viene en binario) y a continuación se intenta almacenar en la base de datos.